<a href="https://colab.research.google.com/github/pko89403/RecsysStudy/blob/master/NeuralCollaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GMF 를 Tensorflow로 구현 

## 사용하는 데이터 
- lastfm 데이터셋 
## Evaluation
- top@K ( K = 10 )


In [ ]:
! ls "/content/drive/My Drive/data"

aclImdb  imdb.tar.gz  index.html


In [ ]:
!wget "http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz" -P "/content/drive/My Drive/data/"

--2020-10-26 05:57:23--  http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz
Resolving mtg.upf.edu (mtg.upf.edu)... 84.89.139.55
Connecting to mtg.upf.edu (mtg.upf.edu)|84.89.139.55|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569202935 (543M) [application/x-gzip]
Saving to: ‘/content/drive/My Drive/data/lastfm-dataset-360K.tar.gz’

lastfm-dataset-360K 100%[===================>] 542.83M  1.25MB/s    in 8m 27s  

2020-10-26 06:05:51 (1.07 MB/s) - ‘/content/drive/My Drive/data/lastfm-dataset-360K.tar.gz’ saved [569202935/569202935]



In [ ]:
!tar -zxvf "/content/drive/My Drive/data/lastfm-dataset-360K.tar.gz" -C "/content/drive/My Drive/data/lastfm/"

lastfm-dataset-360K/
lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
lastfm-dataset-360K/README.txt
lastfm-dataset-360K/mbox_sha1sum.py
lastfm-dataset-360K/usersha1-profile.tsv


In [ ]:
!head "/content/drive/My Drive/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv"

00000c289a1829a808ac09c00daf10bc3c4e223b	3bd73256-3905-4f3a-97e2-8b341527f805	betty blowtorch	2137
00000c289a1829a808ac09c00daf10bc3c4e223b	f2fb0ff0-5679-42ec-a55c-15109ce6e320	die Ärzte	1099
00000c289a1829a808ac09c00daf10bc3c4e223b	b3ae82c2-e60b-4551-a76d-6620f1b456aa	melissa etheridge	897
00000c289a1829a808ac09c00daf10bc3c4e223b	3d6bbeb7-f90e-4d10-b440-e153c0d10b53	elvenking	717
00000c289a1829a808ac09c00daf10bc3c4e223b	bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8	juliette & the licks	706
00000c289a1829a808ac09c00daf10bc3c4e223b	8bfac288-ccc5-448d-9573-c33ea2aa5c30	red hot chili peppers	691
00000c289a1829a808ac09c00daf10bc3c4e223b	6531c8b1-76ea-4141-b270-eb1ac5b41375	magica	545
00000c289a1829a808ac09c00daf10bc3c4e223b	21f3573f-10cf-44b3-aeaa-26cccd8448b5	the black dahlia murder	507
00000c289a1829a808ac09c00daf10bc3c4e223b	c5db90c4-580d-4f33-b364-fbaa5a3a58b5	the murmurs	424
00000c289a1829a808ac09c00daf10bc3c4e223b	0639533a-0402-40ba-b6e0-18b067198b73	lunachicks	403


In [ ]:
import pandas as pd 
import numpy as np
import pickle

,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
0,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
1,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
2,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
3,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
4,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691


In [ ]:
def load_dataset():
  
  
  df = pd.read_csv("/content/drive/My Drive/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv", sep='\t')
  

  df = df.drop(df.columns[1], axis=1)
  df.columns = ['user', 'item', 'plays']
  df = df.dropna()
  df = df.loc[df.plays != 0]


  df_count = df.groupby(['user']).count()
  df['count'] = df.groupby('user')['user'].transform('count')
  df = df[df['count'] > 1]

  
  # Return Series of codes as well as the index.
  df['user_id'] = df['user'].astype('category').cat.codes
  df['item_id'] = df['item'].astype('category').cat.codes  


  item_lookup = df[['item_id', 'item']].drop_duplicates()
  item_lookup['item_id'] = item_lookup.item_id.astype(str)


  user_lookup = df[['user_id', 'user']].drop_duplicates()
  user_lookup['user_id'] = user_lookup.user_id.astype(str)


  df = df[['user_id', 'item_id', 'plays']]
  users = list(np.sort(df.user_id.unique()))
  items = list(np.sort(df.item_id.unique()))


  df_train, df_test = train_test_split(df)


  rows = df_train.user_id.astype(int)
  cols = df_train.item_id.astype(int)

  values = list(df_train.plays)

  uids = np.array(rows.tolist())
  iids = np.array(cols.tolist())


  df_neg = get_negatives(uids, iids, items, df_test)

  return uids, iids, df_train, df_test, df_neg, users, items, item_lookup

In [ ]:
def mask_first(x):
  result = np.ones_like(x)
  result[0] = 0
  return result

In [ ]:
def train_test_split(df):
  df_test = df.copy(deep=True)
  df_train = df.copy(deep=True)

  # Group by user and select only the first item for each user 
  df_test = df_test.groupby(['user_id']).first()
  df_test['user_id'] = df_test.index
  df_test = df_test[['user_id', 'item_id', 'plays']]
  print(df_test.index.name)
  df_test.rename(index={'name': ''}, inplace=True)

  # Remove the same items as we for our test set in our training set
  mask = df.groupby(['user_id'])['user_id'].transform(mask_first).astype(bool)
  df_train = df.loc[mask]

  return df_train, df_test

In [ ]:
def get_negatives(uids, iids, items, df_test):
  negativeList = []
  test_u = df_test['user_id'].values.tolist()
  test_i = df_test['item_id'].values.tolist()

  test_ratings = list(zip(test_u, test_i))
  zipped = set(zip(uids, iids))

  for (u, i) in test_ratings:
    negatives = []
    negatives.append((u, i))
    for t in range(100):
      j = np.random.randint(len(items)) # Get random item id
      while (u, j) in zipped: # Check if there is an interaction
        j = np.random.randint(len(items)) # If yes, generate a new item id 
      negatives.append(j) # Once a negative interaction is found we add it
    negativeList.append(negatives)

  df_neg = pd.DataFrame(negativeList)

  return df_neg

In [ ]:
def get_train_instances():
  user_input, item_input, labels = [], [], []
  zipped = set(zip(uids, iids))

  for (u, i) in zip(uids, iids):
    # Add positive interaction
    user_input.append(u)
    item_input.append(i)
    labels.append(1)

    # Sample random negative interaction
    for t in range(num_neg):
      j = np.random.randint(len(items))
      while (u, i) in zipped:
        j = np.random.randint(len(items))
      
      user_input.append(u)
      item_input.append(j)
      labels.append(0)

    return user_input, item_input, labels

In [ ]:
def random_mini_batches(U, I, L, mini_batch_size=256):
  mini_batches = []
  
  shuffled_U, shuffled_I, shuffled_L = shuffle(U, I, L)

  num_complete_batches = int(math.floor(len(U)/mini_batch_size))
  for k in range(0, num_complete_batches):
    mini_batch_U = shuffled_U[k * mini_batch_size : k * mini_batch_size + mini_batch_size]
    mini_batch_I = shuffled_I[k * mini_batch_size : k * mini_batch_size + mini_batch_size]
    mini_batch_L = shuffled_L[k * mini_batch_size : k * mini_batch_size + mini_batch_size]

    mini_batch = (mini_batch_U, mini_batch_I, mini_batch_L)
    mini_batches.append(mini_batch)
  
  if len(U) % mini_batch_size != 0:
    mini_batch_U = shuffled_U[num_complete_batches * mini_batch_size : len(U)]
    mini_batch_I = shuffled_I[num_complete_batches * mini_batch_size : len(U)]
    mini_batch_L = shuffled_L[num_complete_batches * mini_batch_size : len(U)]

    mini_batch = (mini_batch_U, mini_batch_I, mini_batch_L)
    mini_batches.append(mini_batch)

  return mini_batches 

In [ ]:
def get_hits(k_ranked, holdout):
  for item in h_ranked:
    if item == holdout:
      return 1
    return 0

In [ ]:
def eval_rating(idx, test_ratings, test_negatives, K):
  map_item_score = {}

  # get the negative interactions our userr
  items = test_negatives[idx]
  # Get the user idx
  user_idx = test_ratings[idx][0]
  # Get the item idx -> holdout item
  holdout = test_ratings[idx][1]

  # Add the holdout to the end of the negative interactions list.
  items.append(holdout)

  # Prepare our user and item arrays for tensorflow
  predict_user = np.full(len(items), user_idx, dtype='int32').reshape(-1,1)
  np_items = np.array(items).reshape(-1, 1)

  # Feed user and items into the TF graph.
  predictions = session.run([output_layer], feed_dict={user: predict_user, item: np_items})

  # Get the predicted score to item id 
  for i in range(len(items)):
    current_item = items[i]
    map_item_score[current_item] = predictions[i]
  
  # Get the K highest ranked items as a list
  h_ranked = heapq.nlargest(K, map_item_score, key=map_item_score.get)

  # Get a list of hit or no hit.
  hits = get_hits(h_ranked, holdout)

  return hits 


In [ ]:
def evaluate(df_neg, K=10):
  hits = []

  test_u = df_test['user_id'].values.tolist()
  test_i = df_test['item_id'].values.tolist()

  test_ratings = list(zip(test_u, test_i))

  df_neg = df_neg.drop(df_neg.columns[0], axis=1)
  test_negatives = df_neg.values.tolist()

  for idx in range(len(test_ratings)):
    hitrate = eval_rating(idx, test_ratings, test_negatives, K)
    hits.append(hitrate)

  return hits

In [ ]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import math
import heapq
from tqdm import tqdm 

In [ ]:
# Load and prepare our data
uids, iids, df_train, df_test, df_neg, users, items, item_lookup = load_dataset()

user_id


In [ ]:
# Hyper-Parameters
num_neg = 4
epoch = 10
batch_size = 256
learning_rate = 1e-3

In [4]:
# Set up Tensorflow Graph
graph = tf.Graph()

with graph.as_default():
  # Define input placeholders for user, item and label
  user = tf.placeholder(tf.int32, shape=(None, 1))
  item = tf.placeholder(tf.int32, shape=(None, 1))
  label = tf.placeholder(tf.int32, shape=(None, 1))

  # User Feature Embedding
  u_var = tf.Variable(tf.random_normal([len(users), 32], stddev=0.05), name='user_embedding')
  user_embedding = tf.nn.embedding_lookup(u_var, user)

  # Item Feature Embedding
  i_var = tf.Variable(tf.random_normal([len(items), 32], stddev=0.05), name='item_embedding')
  item_embedding = tf.nn.embedding_lookup(i_var, item)

  # Flatten Our User and Item Embedding
  user_embedding = tf.keras.layers.Flatten()(user_embedding)
  item_embedding = tf.keras.layers.Flatten()(item_embedding)

  # Concatenate our two embedding vectors together
  concatenated = tf.keras.layers.concatenate([user_embedding, item_embedding])

  # Add a first dropout layer
  dropout = tf.keras.layers.Dropout(0.2)(concatenated)

  # Below we add our four hidden layers along with batch
  # Normalization and Dropout. We use relu as the Activation Funtion
  layer_1 = tf.keras.layers.Dense(64, activation='relu', name='layer1')(dropout)
  batch_norm1 = tf.keras.layers.BatchNormalization(name='batch_norm1')(layer_1)
  dropout1 = tf.keras.layers.Dropout(0.2, name='dropout1')(batch_norm1)

  layer_2 = tf.keras.layers.Dense(32, activation='relu', name='layer2')(dropout1)
  batch_norm2 = tf.keras.layers.BatchNormalization(name='batch_norm2')(layer_2)
  dropout2 = tf.keras.layers.Dropout(0.2, name='dropout2')(batch_norm2)

  layer_3 = tf.keras.layers.Dense(16, activation='relu', name='layer3')(dropout2)
  layer_4 = tf.keras.layers.Dense(8, activation='relu', name='layer4')(layer_3)

  # Our final single neuron output layer
  output_layer = tf.keras.layers.Dense(1,
    kernel_initializer="lecun_uniform",
    name='output_layer')(layer_4)

  # Define Loss Function as Cross Entropy
  labels = tf.cast(label, tf.float32)
  logits = output_layer
  loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    labels=labels,
    logits=logits))
  
  # Train using the Adam optimizer to minimize our loss 
  opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
  step = opt.minimize(loss)

  # Initialize all tensorflow variables
  init = tf.global_variables_initializer()

session = tf.session(config=None, graph=graph)
session.run(init)


NameError: ignored

In [ ]:

for epoch in range(epochs):
  # Get our training input.
  user_input, item_input, labels = get_train_instances()

  # Generate a list of minibatches
  minibatches = random_mini_batches(user_input, item_input, labels)

  # This has nothing to do with tensorflow but gives
  # us a nice progress bar for the training 
  progress = tqdm(total=len(minibatches))

  # Loop over each batch and feed our users, items and labels
  # into our graph.
  for minibatch in minibatches:
    feed_dict = {user: np.array(minibatch[0]).reshape(-1,1),
                 item: np.array(minibatch[1]).reshape(-1,1),
                 label: np.array(minibatch[2]).reshape(-1,1)}

    _, l = session.run([step, loss], feed_dict)

    # Update the progress 
    progress.update(1)
    progress.set_description('Epoch: %d - Loss: %.3f' % (epoch+1, l))
  
  progress.close()

  # Calculate top@K
  hits = evaluate(df_neg)
  print(np.array(hits).mean())